In [1]:
# COLAB
from google.colab import drive
drive.mount('/content/drive')

# COMMON
import numpy as np
import torch

# SEED
import random
import os

# DATA
import pandas as pd

# PREPROCESS DATA
import re
!pip install contractions
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

nltk.download('punkt')
nltk.download('stopwords')

# DATASET/LOADER
!pip install datasets
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

# MODEL
from transformers import AutoModelForSequenceClassification

# TRAIN
from tqdm.auto import tqdm
import torch.optim as optim
import torch.nn as nn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# define variable
MODEL_CHECKPOINT = "bert-base-cased"
BATCH_SIZE = 32
MAX_LENGTH = 350
NUM_EPOCHS = 3
LEARNING_RATE = 5e-5

# move model to device if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# set seed
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed()

# Prepare Data

In [18]:
# train data path
train_data_path = "/content/drive/MyDrive/nlu/data/train.csv"

# development data path
dev_data_path = "/content/drive/MyDrive/nlu/data/dev.csv"

# read data
train_data = pd.read_csv(train_data_path)
dev_data = pd.read_csv(dev_data_path)

In [19]:
train_data

,text_1,text_2,label
0,Nick ( Kevin Anderson ) goes back to his homet...,Bank clerk Miles Cullen ( Elliott Gould ) is r...,1
1,"Kate Nelligan , always a forthright and grippi...",Anthony Perkins reportedly felt threatened as ...,1
2,"Patrick, Please, contact Zimin Lu, 713 853 638...","Corey, Paula West Trilium Court 107 The Woodla...",1
3,"wow, ok so my mom was saying how when she gets...","dude, i'm cold",1
4,"Flat broke again , Stan borrows from ' Basher ...",I had been underwhelmed by my first viewing of...,0
...,...,...,...
29995,"John and Krishna, I am sending you an outline ...",FYI Vince,1
29996,You want to know about rabies ? Try this : One...,Wow what a great idea for a movie getting a bu...,1
29997,Plz see email. Sent you info for u & Rick. Bes...,Here's Paul's memo with Mona's edits and a cou...,1
29998,Pretty blonde Paris Hilton ( as Victoria Engli...,Crippled violinist David Miles ( as Filippo ) ...,1


# Preprocessing

In [2]:
# get preprocessed data if it is saved
train_data = pd.read_csv("/content/drive/MyDrive/nlu/data/preprocess_train.csv")
dev_data = pd.read_csv("/content/drive/MyDrive/nlu/data/preprocess_dev.csv")

In [45]:
# remove null row
train_data.drop(train_data[train_data.isnull().any(axis=1)].index, inplace=True)
dev_data.drop(dev_data[dev_data.isnull().any(axis=1)].index, inplace=True)        # need to predict the data including Nan?

In [4]:
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

# define preprocessing function
def preprocess_text(text):
    # convert to lower case
    text = text.lower()

    # expand contractions (ex. don't -> do not)
    text = contractions.fix(text)

    # remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # remove multiple spaces
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # remove stopwords
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(tokens)

    # lemmatize
    tokens = []
    spacy_string = nlp(text)
    for token in spacy_string:
        tokens.append(token.lemma_)
    text = " ".join(tokens)

    return text

In [5]:
# preprocess train data
train_data['text_1'] = train_data['text_1'].apply(preprocess_text)
train_data['text_2'] = train_data['text_2'].apply(preprocess_text)

# preprocess development data
dev_data['text_1'] = dev_data['text_1'].apply(preprocess_text)
dev_data['text_2'] = dev_data['text_2'].apply(preprocess_text)

In [46]:
# save preprocessed data
train_data.to_csv("/content/drive/MyDrive/nlu/data/preprocess_train.csv", index=False)
dev_data.to_csv("/content/drive/MyDrive/nlu/data/preprocess_dev.csv", index=False)

In [3]:
# check max length of text and average text length for later tokenization
max_len_1 = 0
max_len_2 = 0
avg_len_1 = 0
avg_len_2 = 0

for i, j in zip(train_data['text_1'], train_data['text_2']):
    avg_len_1 += len(i)
    avg_len_2 += len(j)

    if len(i) > max_len_1:
        max_len_1 = len(i)
    if len(j) > max_len_2:
        max_len_2 = len(j)

max_len_1, max_len_2, avg_len_1 / len(train_data['text_1']), avg_len_2 / len(train_data['text_2'])

(4480, 2114, 373.35234686025893, 372.8583854671975)

# Data Loader

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def preprocess_function(records):
    return tokenizer(records['text_1'], records['text_2'], truncation=True, return_token_type_ids=True, max_length = MAX_LENGTH)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# own dataset
class PairwiseDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        dataset = Dataset.from_pandas(data)
        # dataset = dataset.remove_columns('__index_level_0__')
        encoded_dataset = dataset.map(preprocess_function, batched=True)

        self.input_ids = encoded_dataset["input_ids"]
        self.token_type_ids = encoded_dataset["token_type_ids"]
        self.attention_mask = encoded_dataset["attention_mask"]
        self.labels = encoded_dataset["label"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'token_type_ids': torch.tensor(self.token_type_ids[idx]),
            'attention_mask': torch.tensor(self.attention_mask[idx]),
            'label': torch.tensor(self.labels[idx])
        }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# prepare dataset
train_dataset = PairwiseDataset(train_data)
dev_dataset = PairwiseDataset(dev_data)

# define dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator
)
dev_dataloader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator
)

Map:   0%|          | 0/29891 [00:00<?, ? examples/s]

Map:   0%|          | 0/5973 [00:00<?, ? examples/s]

# Model

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training

In [21]:
# train function
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    train_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for batch in tqdm(dataloader):
        # move batch to device
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # zero the gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # calculate loss
        loss = criterion(logits, labels)

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()

        # compute running loss and accuracy
        train_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    # calculate epoch loss and accuracy
    train_loss /= len(dataloader)
    train_accuracy = correct_predictions / total_predictions

    return train_loss, train_accuracy

# test/validation function
def test(model, dataloader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.inference_mode():
        for batch in tqdm(dataloader):
            # move batch to device
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # forward pass
            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # calculate loss
            loss = criterion(logits, labels)

            # compute running loss and accuracy
            test_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    # calculate epoch loss and accuracy
    test_loss /= len(dataloader)
    test_accuracy = correct_predictions / total_predictions

    return test_loss, test_accuracy

In [ ]:
# move model to device if GPU is available
model.to(device)

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# define loss function
criterion = nn.CrossEntropyLoss()

# result
res = {"train_loss": [], "train_accuracy": [], "dev_loss": [], "dev_accuracy": []}

# training loop
for epoch in range(NUM_EPOCHS):
    # train
    train_loss, train_accuracy = train(model=model, dataloader=train_dataloader, optimizer=optimizer, criterion=criterion, device=device)

    # validation
    dev_loss, dev_accuracy = test(model=model, dataloader=dev_dataloader, criterion=criterion, device=device)

    # save results
    res["train_loss"].append(train_loss)
    res["train_accuracy"].append(train_accuracy)
    res["dev_loss"].append(dev_loss)
    res["dev_accuracy"].append(dev_accuracy)

    # print result
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | train loss: {train_loss:.4f}, train acc: {train_accuracy:.4f} | dev loss: {dev_loss:.4f}, dev acc: {dev_accuracy:.4f}")

  0%|          | 0/935 [00:00<?, ?it/s]

# Test